# CarRacing 환경을 위한 DQN 학습

이 노트북은 CarRacing-v3 환경을 위한 완전한 DQN 학습 파이프라인을 구현합니다. Deep Q-Networks를 사용하여 자동차를 운전하는 AI 에이전트를 학습시킵니다!

## 수행할 작업
1. **환경 설정** - 전처리가 포함된 CarRacing
2. **DQN 네트워크 구축** - 시각적 입력 처리를 위한 CNN
3. **학습 구성요소 구현** - Replay buffer, target network 등
4. **에이전트 학습** - 운전을 학습하는 과정 관찰!
5. **결과 분석** - 학습 진행 상황 시각화

## 빠른 시작
다음을 확인하세요:
- 가상 환경 활성화
- 모든 종속성 설치
- 튜토리얼 노트북을 먼저 실행 (권장)

레이싱 AI 학습을 시작합시다!

In [ ]:
# 필요한 모든 라이브러리 import
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import gymnasium as gym
import cv2
import random
import os
import time
from collections import deque
from typing import Tuple, List, Optional, Dict, Any
from pathlib import Path
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

# 그래프 스타일 설정
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = [12, 8]

# 장치 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🖥️  사용 장치: {device}")
print(f"🐍 Python 패키지 준비 완료!")

# 재현 가능성을 위한 랜덤 시드 설정
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
print(f"🎲 재현 가능성을 위한 랜덤 시드 설정 완료")

---
## 하이퍼파라미터 설정

이 하이퍼파라미터들은 학습 프로세스를 제어합니다. 다양한 값으로 실험해보세요!

In [ ]:
# 하이퍼파라미터 - 실험을 위해 이 값들을 수정해보세요!
HYPERPARAMETERS = {
    # 학습 파라미터
    'learning_rate': 0.0001,
    'gamma': 0.99,  # 할인 인자
    
    # 탐험 파라미터
    'epsilon_start': 1.0,
    'epsilon_end': 0.01,
    'epsilon_decay': 0.995,
    
    # 학습 파라미터
    'batch_size': 32,
    'buffer_size': 10000,
    'target_update': 1000,  # N 스텝마다 target network 업데이트
    
    # 에피소드 파라미터
    'num_episodes': 100,  # 노트북용으로 적은 에피소드로 시작
    'max_steps_per_episode': 1000,
    
    # 환경 파라미터
    'frame_stack': 4,
    'image_size': (84, 84),
    
    # 로깅
    'save_interval': 25,
    'log_interval': 5
}

print("📊 하이퍼파라미터:")
for key, value in HYPERPARAMETERS.items():
    print(f"  {key}: {value}")
    
print(f"\n💡 팁: 더 나은 성능을 위해 'num_episodes'를 500+로 증가시키세요!")

---
## 환경 설정

적절한 전처리와 함께 CarRacing 환경을 생성합니다.

In [ ]:
class CarRacingWrapper:
    """전처리가 포함된 CarRacing 환경 래퍼"""
    
    def __init__(self, render_mode: Optional[str] = None):
        """
        CarRacing 환경 래퍼 초기화
        
        Args:
            render_mode: 렌더링 모드 ('human', 'rgb_array', 또는 None)
        """
        self.env = gym.make('CarRacing-v3', render_mode=render_mode)
        self.frame_stack = HYPERPARAMETERS['frame_stack']
        self.image_size = HYPERPARAMETERS['image_size']
        
        # 프레임 스태킹을 위한 버퍼
        self.frames = deque(maxlen=self.frame_stack)
        
    def reset(self) -> np.ndarray:
        """환경 리셋 및 초기 스택 프레임 반환"""
        obs, info = self.env.reset()
        
        # 초기 프레임 전처리
        processed_frame = self._preprocess_frame(obs)
        
        # 첫 프레임을 반복하여 프레임 스택 초기화
        for _ in range(self.frame_stack):
            self.frames.append(processed_frame)
            
        return self._get_stacked_frames()
        
    def step(self, action: int) -> Tuple[np.ndarray, float, bool, bool, Dict]:
        """
        행동을 수행하고 전처리된 관측값 반환
        
        Args:
            action: 이산 행동 인덱스
            
        Returns:
            (관측값, 보상, 종료여부, 잘림여부, 정보) 튜플
        """
        # 이산 행동을 연속 행동으로 변환
        continuous_action = self._discrete_to_continuous(action)
        
        # 환경에서 스텝 수행
        obs, reward, terminated, truncated, info = self.env.step(continuous_action)
        
        # 프레임 전처리 및 스택
        processed_frame = self._preprocess_frame(obs)
        self.frames.append(processed_frame)
        stacked_frames = self._get_stacked_frames()
        
        return stacked_frames, reward, terminated, truncated, info
        
    def _preprocess_frame(self, frame: np.ndarray) -> np.ndarray:
        """프레임 전처리: 크기 조정, 그레이스케일, 정규화"""
        # 그레이스케일로 변환
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        
        # 타겟 크기로 리사이즈
        resized_frame = cv2.resize(gray_frame, self.image_size)
        
        # [0, 1] 범위로 정규화
        normalized_frame = resized_frame.astype(np.float32) / 255.0
        
        return normalized_frame
        
    def _get_stacked_frames(self) -> np.ndarray:
        """스택된 프레임을 numpy 배열로 반환"""
        return np.array(list(self.frames))
        
    def _discrete_to_continuous(self, action: int) -> np.ndarray:
        """이산 행동을 연속 행동 공간으로 변환"""
        if action == 0:     # 왼쪽 회전
            return np.array([-0.5, 0.3, 0.0])
        elif action == 1:   # 직진
            return np.array([0.0, 0.5, 0.0])
        elif action == 2:   # 오른쪽 회전
            return np.array([0.5, 0.3, 0.0])
        elif action == 3:   # 브레이크
            return np.array([0.0, 0.0, 0.8])
        else:
            return np.array([0.0, 0.0, 0.0])
            
    def close(self):
        """환경 종료"""
        self.env.close()

# 환경 테스트
print("🚗 CarRacing 환경 테스트 중...")
test_env = CarRacingWrapper()
test_obs = test_env.reset()
print(f"✅ 환경 초기화 성공!")
print(f"   관측값 형태: {test_obs.shape}")
print(f"   행동 공간: 4개 이산 행동 (왼쪽, 직진, 오른쪽, 브레이크)")
test_env.close()
del test_env

---
## DQN 네트워크 구조

CNN 기반 Deep Q-Network를 구축합니다!

In [ ]:
class DQN(nn.Module):
    """CarRacing을 위한 CNN 기반 Deep Q-Network"""
    
    def __init__(self, action_dim: int = 4, input_channels: int = 4):
        """
        DQN 네트워크 초기화
        
        Args:
            action_dim: 이산 행동의 개수
            input_channels: 입력 채널 수 (프레임 스택)
        """
        super(DQN, self).__init__()
        
        # 합성곱 레이어
        self.conv1 = nn.Conv2d(input_channels, 32, kernel_size=8, stride=4, padding=0)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=0)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0)
        
        # Conv 출력 크기 계산
        self._conv_output_size = self._get_conv_output_size((input_channels, 84, 84))
        
        # 완전 연결 레이어
        self.fc1 = nn.Linear(self._conv_output_size, 512)
        self.fc2 = nn.Linear(512, action_dim)
        
        # 가중치 초기화
        self._initialize_weights()
        
    def _get_conv_output_size(self, input_shape: Tuple[int, int, int]) -> int:
        """Conv 레이어 통과 후 출력 크기 계산"""
        with torch.no_grad():
            dummy_input = torch.zeros(1, *input_shape)
            dummy_output = self._forward_conv(dummy_input)
            return dummy_output.numel()
            
    def _forward_conv(self, x: torch.Tensor) -> torch.Tensor:
        """Conv 레이어만 통과하는 forward pass"""
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        return x.view(x.size(0), -1)
        
    def _initialize_weights(self):
        """네트워크 가중치 초기화"""
        for module in self.modules():
            if isinstance(module, (nn.Conv2d, nn.Linear)):
                nn.init.xavier_uniform_(module.weight)
                if module.bias is not None:
                    nn.init.constant_(module.bias, 0)
                    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """네트워크 forward pass"""
        x = self._forward_conv(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 네트워크 생성 및 분석
dqn = DQN(action_dim=4, input_channels=4).to(device)
total_params = sum(p.numel() for p in dqn.parameters() if p.requires_grad)

print("🧠 DQN 네트워크 생성 완료!")
print(f"   총 파라미터 수: {total_params:,}")
print(f"   네트워크 크기: ~{total_params * 4 / 1024 / 1024:.1f} MB")

# Forward pass 테스트
dummy_input = torch.randn(1, 4, 84, 84).to(device)
with torch.no_grad():
    output = dqn(dummy_input)
print(f"   입력 형태: {dummy_input.shape}")
print(f"   출력 형태: {output.shape}")
print(f"✅ 네트워크 테스트 통과!")

---
## Experience Replay Buffer

Replay buffer는 학습을 위한 경험을 저장하고 샘플링합니다.

In [ ]:
class ReplayBuffer:
    """전환을 저장하기 위한 경험 재생 버퍼"""
    
    def __init__(self, capacity: int):
        self.buffer = deque(maxlen=capacity)
        self.capacity = capacity
        
    def push(self, state, action, reward, next_state, done):
        """버퍼에 전환 추가"""
        self.buffer.append((state, action, reward, next_state, done))
        
    def sample(self, batch_size: int) -> Tuple:
        """전환 배치 샘플링"""
        batch = random.sample(self.buffer, batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)
        
        return (
            torch.FloatTensor(np.array(states)),
            torch.LongTensor(actions),
            torch.FloatTensor(rewards),
            torch.FloatTensor(np.array(next_states)),
            torch.BoolTensor(dones)
        )
        
    def __len__(self):
        return len(self.buffer)

# Replay buffer 생성
replay_buffer = ReplayBuffer(HYPERPARAMETERS['buffer_size'])
print(f"🗃️  Replay buffer 생성 완료 (용량: {HYPERPARAMETERS['buffer_size']:,})")

---
## DQN 에이전트

모든 구성요소를 결합한 DQN 에이전트를 생성합니다!

In [ ]:
class DQNAgent:
    """모든 학습 구성요소를 포함한 DQN 에이전트"""
    
    def __init__(self, device: torch.device):
        self.device = device
        self.action_dim = 4  # 왼쪽, 직진, 오른쪽, 브레이크
        
        # 네트워크
        self.main_network = DQN(self.action_dim).to(device)
        self.target_network = DQN(self.action_dim).to(device)
        self.target_network.load_state_dict(self.main_network.state_dict())
        
        # 옵티마이저
        self.optimizer = optim.Adam(
            self.main_network.parameters(), 
            lr=HYPERPARAMETERS['learning_rate']
        )
        
        # Replay buffer
        self.replay_buffer = ReplayBuffer(HYPERPARAMETERS['buffer_size'])
        
        # 탐험 전략
        self.epsilon = HYPERPARAMETERS['epsilon_start']
        self.epsilon_decay = HYPERPARAMETERS['epsilon_decay']
        self.epsilon_min = HYPERPARAMETERS['epsilon_end']
        
        # 학습 카운터
        self.step_count = 0
        self.episode_count = 0
        
    def select_action(self, state: np.ndarray, training: bool = True) -> int:
        """Epsilon-greedy 정책으로 행동 선택"""
        if training and random.random() < self.epsilon:
            return random.randint(0, self.action_dim - 1)
        
        with torch.no_grad():
            state_tensor = torch.FloatTensor(state).unsqueeze(0).to(self.device)
            q_values = self.main_network(state_tensor)
            return q_values.argmax().item()
            
    def store_transition(self, state, action, reward, next_state, done):
        """Replay buffer에 전환 저장"""
        self.replay_buffer.push(state, action, reward, next_state, done)
        
    def update(self) -> Optional[float]:
        """Replay buffer의 배치를 사용하여 네트워크 업데이트"""
        if len(self.replay_buffer) < HYPERPARAMETERS['batch_size']:
            return None
            
        # 배치 샘플링
        states, actions, rewards, next_states, dones = \
            self.replay_buffer.sample(HYPERPARAMETERS['batch_size'])
            
        states = states.to(self.device)
        actions = actions.to(self.device)
        rewards = rewards.to(self.device)
        next_states = next_states.to(self.device)
        dones = dones.to(self.device)
        
        # 현재 Q-값
        current_q_values = self.main_network(states).gather(1, actions.unsqueeze(1))
        
        # Target network의 다음 Q-값
        with torch.no_grad():
            next_q_values = self.target_network(next_states).max(1)[0]
            targets = rewards + (HYPERPARAMETERS['gamma'] * next_q_values * (~dones))
            
        # 손실 계산
        loss = F.smooth_l1_loss(current_q_values.squeeze(), targets)
        
        # 최적화
        self.optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.main_network.parameters(), 1.0)
        self.optimizer.step()
        
        # 스텝 카운터 업데이트
        self.step_count += 1
        
        # Target network 업데이트
        if self.step_count % HYPERPARAMETERS['target_update'] == 0:
            self.update_target_network()
            
        return loss.item()
        
    def update_target_network(self):
        """Main network 가중치로 target network 업데이트"""
        self.target_network.load_state_dict(self.main_network.state_dict())
        
    def update_epsilon(self):
        """다음 에피소드를 위한 epsilon 업데이트"""
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)
        self.episode_count += 1

# 에이전트 생성
agent = DQNAgent(device)
print(f"🤖 DQN 에이전트 생성 완료!")
print(f"   Main network 파라미터: {sum(p.numel() for p in agent.main_network.parameters()):,}")
print(f"   Target network 파라미터: {sum(p.numel() for p in agent.target_network.parameters()):,}")
print(f"   초기 epsilon: {agent.epsilon}")

---
## 학습 루프

이제 에이전트를 학습시킵니다! 여기서 마법이 일어납니다.

In [ ]:
def train_agent(num_episodes: int = HYPERPARAMETERS['num_episodes']):
    """DQN 에이전트 학습"""
    
    # 환경 초기화
    env = CarRacingWrapper()
    
    # 학습 통계
    episode_rewards = []
    episode_losses = []
    episode_lengths = []
    
    # 저장용 디렉토리 생성
    models_dir = Path("../models/saved_weights")
    models_dir.mkdir(parents=True, exist_ok=True)
    
    print(f"🚀 {num_episodes} 에피소드 학습 시작...")
    print(f"📊 {HYPERPARAMETERS['log_interval']} 에피소드마다 진행상황 표시")
    print("-" * 60)
    
    start_time = time.time()
    best_reward = float('-inf')
    
    # 진행바와 함께 학습 루프
    progress_bar = tqdm(range(num_episodes), desc="학습 중")
    
    for episode in progress_bar:
        # 환경 리셋
        state = env.reset()
        episode_reward = 0.0
        episode_loss_list = []
        step = 0
        
        # 에피소드 루프
        for step in range(HYPERPARAMETERS['max_steps_per_episode']):
            # 행동 선택 및 수행
            action = agent.select_action(state, training=True)
            next_state, reward, terminated, truncated, _ = env.step(action)
            
            # 전환 저장
            done = terminated or truncated
            agent.store_transition(state, action, reward, next_state, done)
            
            # 에이전트 업데이트
            loss = agent.update()
            if loss is not None:
                episode_loss_list.append(loss)
                
            # 상태 및 보상 업데이트
            state = next_state
            episode_reward += reward
            
            if done:
                break
                
        # 통계 업데이트
        episode_rewards.append(episode_reward)
        episode_lengths.append(step + 1)
        avg_loss = np.mean(episode_loss_list) if episode_loss_list else 0.0
        episode_losses.append(avg_loss)
        
        # 탐험 업데이트
        agent.update_epsilon()
        
        # 진행바 업데이트
        recent_rewards = episode_rewards[-10:] if len(episode_rewards) >= 10 else episode_rewards
        avg_reward = np.mean(recent_rewards)
        progress_bar.set_postfix({
            '보상': f'{episode_reward:.1f}',
            '평균': f'{avg_reward:.1f}',
            'ε': f'{agent.epsilon:.3f}'
        })
        
        # 로깅
        if episode % HYPERPARAMETERS['log_interval'] == 0 and episode > 0:
            print(f"\n에피소드 {episode:4d} | "
                  f"보상: {episode_reward:8.2f} | "
                  f"평균 보상: {avg_reward:8.2f} | "
                  f"손실: {avg_loss:.4f} | "
                  f"Epsilon: {agent.epsilon:.4f} | "
                  f"버퍼: {len(agent.replay_buffer)}")
                  
        # 모델 저장
        if episode % HYPERPARAMETERS['save_interval'] == 0 and episode > 0:
            model_path = models_dir / f"dqn_episode_{episode}.pth"
            torch.save({
                'main_network': agent.main_network.state_dict(),
                'target_network': agent.target_network.state_dict(),
                'optimizer': agent.optimizer.state_dict(),
                'epsilon': agent.epsilon,
                'step_count': agent.step_count,
                'episode_count': agent.episode_count
            }, model_path)
            
            # 최고 모델 저장
            if episode_reward > best_reward:
                best_reward = episode_reward
                best_model_path = models_dir / "dqn_best.pth"
                torch.save({
                    'main_network': agent.main_network.state_dict(),
                    'target_network': agent.target_network.state_dict(),
                    'optimizer': agent.optimizer.state_dict(),
                    'epsilon': agent.epsilon,
                    'step_count': agent.step_count,
                    'episode_count': agent.episode_count
                }, best_model_path)
                print(f"💾 새로운 최고 모델 저장! 보상: {best_reward:.2f}")
    
    # 최종 모델 저장
    final_model_path = models_dir / "dqn_final.pth"
    torch.save({
        'main_network': agent.main_network.state_dict(),
        'target_network': agent.target_network.state_dict(),
        'optimizer': agent.optimizer.state_dict(),
        'epsilon': agent.epsilon,
        'step_count': agent.step_count,
        'episode_count': agent.episode_count
    }, final_model_path)
    
    # 학습 요약
    total_time = time.time() - start_time
    print("\n" + "=" * 60)
    print("🎉 학습 완료!")
    print("=" * 60)
    print(f"총 에피소드: {len(episode_rewards)}")
    print(f"총 시간: {total_time/60:.1f} 분")
    print(f"평균 보상: {np.mean(episode_rewards):.2f}")
    print(f"최고 보상: {np.max(episode_rewards):.2f}")
    print(f"최종 epsilon: {agent.epsilon:.4f}")
    print(f"총 스텝: {agent.step_count}")
    
    # 정리
    env.close()
    
    return episode_rewards, episode_losses, episode_lengths

# 학습 시작!
print("🏁 학습 준비 완료!")
print(f"📋 {HYPERPARAMETERS['num_episodes']} 에피소드 학습 예정")
print(f"⚡ 사용 장치: {device}")

In [ ]:
# 학습 실행!
episode_rewards, episode_losses, episode_lengths = train_agent()

print("\n🎊 학습 완료! 아래 결과를 확인하세요.")

# ---
# ## 📊 학습 결과 분석
# 
# 에이전트의 학습 성과를 시각화해 봅시다!

In [ ]:
# 종합적인 학습 결과 그래프 생성
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('🏎️ DQN 학습 결과', fontsize=16, fontweight='bold')

# 에피소드 보상
axes[0, 0].plot(episode_rewards, 'b-', alpha=0.6)
if len(episode_rewards) >= 10:
    # 이동평균 추가
    window = min(10, len(episode_rewards))
    moving_avg = np.convolve(episode_rewards, np.ones(window)/window, mode='valid')
    axes[0, 0].plot(range(window-1, len(episode_rewards)), moving_avg, 'r-', linewidth=2, label=f'이동평균({window})')
    axes[0, 0].legend()
axes[0, 0].set_title('에피소드 보상')
axes[0, 0].set_xlabel('에피소드')
axes[0, 0].set_ylabel('보상')
axes[0, 0].grid(True, alpha=0.3)

# 에피소드 손실
non_zero_losses = [loss for loss in episode_losses if loss > 0]
if non_zero_losses:
    axes[0, 1].plot(non_zero_losses, 'g-')
    axes[0, 1].set_title('학습 손실')
    axes[0, 1].set_xlabel('에피소드 (학습 포함)')
    axes[0, 1].set_ylabel('손실')
    axes[0, 1].grid(True, alpha=0.3)
else:
    axes[0, 1].text(0.5, 0.5, '학습 데이터 없음\n(버퍼가 너무 작음)', 
                   ha='center', va='center', transform=axes[0, 1].transAxes)
    axes[0, 1].set_title('학습 손실')

# 에피소드 길이
axes[1, 0].plot(episode_lengths, 'orange')
axes[1, 0].set_title('에피소드 길이')
axes[1, 0].set_xlabel('에피소드')
axes[1, 0].set_ylabel('스텝')
axes[1, 0].grid(True, alpha=0.3)

# Epsilon 감소
epsilons = [HYPERPARAMETERS['epsilon_start'] * (HYPERPARAMETERS['epsilon_decay'] ** i) for i in range(len(episode_rewards))]
epsilons = [max(HYPERPARAMETERS['epsilon_end'], eps) for eps in epsilons]
axes[1, 1].plot(epsilons, 'purple')
axes[1, 1].set_title('Epsilon 감소')
axes[1, 1].set_xlabel('에피소드')
axes[1, 1].set_ylabel('Epsilon')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 통계 출력
print("📈 학습 통계:")
print(f"   에피소드: {len(episode_rewards)}")
print(f"   평균 보상: {np.mean(episode_rewards):.2f} ± {np.std(episode_rewards):.2f}")
print(f"   최고 보상: {np.max(episode_rewards):.2f}")
print(f"   최저 보상: {np.min(episode_rewards):.2f}")
print(f"   평균 에피소드 길이: {np.mean(episode_lengths):.1f} 스텝")
print(f"   최종 epsilon: {agent.epsilon:.4f}")

# 성능 분석
if len(episode_rewards) >= 20:
    early_rewards = np.mean(episode_rewards[:10])
    late_rewards = np.mean(episode_rewards[-10:])
    improvement = late_rewards - early_rewards
    print(f"\n📊 학습 진행상황:")
    print(f"   초기 에피소드 (1-10): {early_rewards:.2f}")
    print(f"   후기 에피소드 ({len(episode_rewards)-9}-{len(episode_rewards)}): {late_rewards:.2f}")
    print(f"   개선도: {improvement:.2f} ({improvement/abs(early_rewards)*100:.1f}%)")
    
    if improvement > 0:
        print("   🎉 에이전트가 학습하고 있습니다!")
    else:
        print("   💡 더 많은 에피소드로 학습하거나 하이퍼파라미터를 조정해보세요")

---
## 🏆 다음 단계

축하합니다! DQN 에이전트를 성공적으로 학습시켰습니다. 다음에 할 수 있는 것들입니다:

### 🎮 에이전트 테스트
학습된 에이전트의 실제 동작을 확인하려면 데모 스크립트를 실행하세요:
```bash
python ../games/demo_trained_agent.py
```

### 🔧 성능 개선
더 나은 결과를 얻기 위해 다음 기법들을 시도해보세요:

1. **더 오래 학습**: `num_episodes`를 500-1000으로 증가
2. **하이퍼파라미터 조정**: 
   - 낮은 학습률 (0.00005)
   - 더 큰 버퍼 크기 (50000)
   - 다른 epsilon 감소율 (0.999)
3. **고급 기법**:
   - Double DQN
   - Dueling DQN
   - Prioritized Experience Replay

### 📚 더 알아보기
- 원본 [DQN 논문](https://arxiv.org/abs/1312.5602) 읽기
- Gymnasium의 다른 환경들 시도
- DQN 변형 구현

### 💾 작업 저장
학습된 모델들이 `../models/saved_weights/`에 저장됩니다:
- `dqn_best.pth` - 최고 성능 모델
- `dqn_final.pth` - 학습 완료 후 최종 모델
- `dqn_episode_X.pth` - 학습 중 체크포인트

즐거운 레이싱 되세요! 🏎️💨

In [ ]:
# 최종 요약
print("🎉 DQN 학습 노트북 완료!")
print("="*50)
print("✅ 환경 설정")
print("✅ DQN 네트워크 구조")
print("✅ 경험 재생 버퍼")
print("✅ 에이전트 학습")
print("✅ 결과 시각화")
print("\n🚀 AI 에이전트가 레이싱 준비 완료!")
print("데모 스크립트를 실행하여 실제 동작을 확인하세요.")